In [1]:
import pandas as pd
import pathlib
import os
from tqdm import tqdm

PROJECT = "gs-mrd"

maindir = "/media/hieunguyen/GSHD_HN01"
path_to_storage = os.path.join(maindir, "storage")
path_to_main_src = "/media/hieunguyen/HNSD01/src/gs-mrd"
path_to_save_merge_feature = os.path.join(path_to_main_src, "all_samples")
os.system(f"mkdir -p {path_to_save_merge_feature}")


0

# Read the metadata, convert labcode


In [3]:
##### modify metadata
if os.path.isfile("All Samples GW_MRD_010924.modified.xlsx") == False:
    metadata = pd.read_excel("All Samples GW_MRD_010924.xlsx")
    '''
    These labcodes have been changed and need to be converted in the metadata.
    '''
    convert_labcode = {
        "HMAAAA03": "ZTKL01A",
        "HMAAAA26": "ZTKL05A",
        "HMAAAA21": "ZTKL07A",
        "ZMC031A": "ZMC031",
        "ZMC057A": "ZMC057",
        "ZMC005A": "ZMC005",
        "ZMG093A": "ZMG093",
        "MDCAAA03": "MQCAAA03",
        "MDAAAA18": "MQAAAA18",
        "ZMG040A": "ZMC040A"
    }
    metadata["SampleID"] = metadata["SampleID"].apply(lambda x: convert_labcode[x] if x in convert_labcode.keys() else x)
    metadata = metadata[~metadata["SampleID"].duplicated()]
    metadata.to_excel("All Samples GW_MRD_010924.modified.xlsx")
else:
    metadata = pd.read_excel("All Samples GW_MRD_010924.modified.xlsx")
    
metadata = metadata[~metadata["SampleID"].duplicated()]

print(f"Number of samples in metadata: {metadata.shape[0]}")

Number of samples in metadata: 1550
